In [ ]:
import os
import skimage.io
from PIL import Image
import cv2
import argparse
import pywt

import itertools
import numpy as np
import pandas as pd
from scipy.fftpack import fft
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable


from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Activation, Concatenate
from keras import Sequential

from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import SGD,Adam
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

#Different Transfer learning rate/models
#https://keras.io/api/applications/
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
# from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
#train dataset
df_train = pd.read_excel("gasoline.xlsx", header=None) #trainning data
df_train_label = pd.read_excel("gasoline.xlsx", sheet_name=1, header=None)


#data cleaning: remove redundant first column
df_train_label = df_train_label.iloc[: , 1:]

df_train_label.columns =['Class']

In [ ]:
#check shapes and data of dataframes
print(df_train.shape)
print(df_train_label.shape)

In [ ]:
df_train_label.head(5)

In [ ]:
def plot_wavelet(signal, path, scales = np.arange(0.25, 512),

                    waveletname = 'morl', #'cmor',

                   cmap = plt.cm.seismic,

                   title = 'Wavelet Transform (Power Spectrum) of signal',

                   ylabel = 'Scales',

                   xlabel = 'Time'):
    
    
    #decode the signal
    
    time = np.arange(len(signal))
    
    dt = time[1] - time[0]

    [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, dt)

    power = (abs(coefficients)) ** 2

   

    period = 1. / frequencies

    maxVal = power.max()
    
    n = 8

    levels = [maxVal/(2**(n-i)) for i in range(n)]

    contourlevels = np.log2(levels)

    #print(power.min(), power.max())
    
   

    fig, ax = plt.subplots(figsize=(2.24, 2.24), dpi=200)

    im = ax.contourf(time, np.log2(period), np.log2(power), contourlevels, extend='both',cmap=cmap)

   

 

    #ax.set_title(title, fontsize=20)

    #ax.set_ylabel(ylabel, fontsize=18)

    #ax.set_xlabel(xlabel, fontsize=18)

   

   # yticks = 2**np.arange(np.ceil(np.log2(period.min())), np.ceil(np.log2(period.max())))

    #ax.set_yticks(np.log2(yticks))

    #ax.set_yticklabels(yticks)

    ax.invert_yaxis()

    ylim = ax.get_ylim()

    #ax.set_ylim(ylim[0], -1)

   

    #cbar_ax = fig.add_axes([0.95, 0.5, 0.03, 0.25])

    #fig.colorbar(im, cax=cbar_ax, orientation="vertical")
    
    
    plt.ioff()
   
    plt.axis('off')
    ax.axis('off')
    
    plt.savefig(path,  bbox_inches='tight',pad_inches = 0)
    

    #plt.savefig("test.png",  bbox_inches='tight',pad_inches = 0)

 

In [ ]:
train_path = 'scalograms/training/'

In [ ]:
#df_validation_label.columns =['Class']
ids = []
names = []

for row in df_train.itertuples():
    
    image_id = 'image' + str(row.Index)
    name = 'image' + str(row.Index) + '.png'

    ids.append(image_id)
    names.append(name)  
        
        
        
    filepath = train_path + name
        
    plot_wavelet(df_train.iloc[row.Index], filepath)

In [ ]:
img_width = 347
img_height = 338

In [ ]:
def extract_scalogram(path):
    height =  img_height# pixels in length
    width = img_width # pixels in width
  
    imgs = np.empty((0, width, height, 3)) 

    for filename in os.listdir(path):
        
        if filename.endswith(".png"):

            img = Image.open(os.path.join(path, filename)).convert('RGB')

            imgs = np.append(imgs, np.array(img).reshape((1, width, height, 3)), axis=0)
    
    return imgs


In [ ]:
X_train = extract_scalogram(train_path)


from sklearn import preprocessing

df_train_label['Class_int'] = pd.Categorical(df_train_label['Class']).codes

y_train = df_train_label['Class_int']


In [ ]:
y_train=to_categorical(y_train)

print((X_train.shape,y_train.shape))

In [ ]:
def normalize_negative_one(img):
    normalized_input = (img - np.amin(img)) / (np.amax(img) - np.amin(img))
    return 2*normalized_input - 1

In [ ]:
X_train = normalize_negative_one(X_train)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
    test_size=0.20, shuffle = True)

In [ ]:
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_val: ', X_val.shape)
print('y_val: ', y_val.shape)

In [ ]:
#Initializing the hyperparameters
batch_size= 16
epochs=10
learn_rate=.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)

In [ ]:
#Image Data Augmentation -  technique of altering the existing data to create some more data for the model training proces

train_dategen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
train_generator = train_dategen.flow(X_train,  y_train, batch_size= batch_size)

validation_dategen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
validation_generator = validation_dategen.flow(X_val, y_val, batch_size= batch_size)

test_generator = ImageDataGenerator(rescale = 1./255.)


In [ ]:
#Learning Rate Annealer
lrr= ReduceLROnPlateau(monitor='val_accuracy', factor=.01, patience=3, min_lr=1e-5) #change learning rATE  to get the best result

In [ ]:
#Defining the googlenet model
#use resnet - try various versions of resnet
#compare googlenet to resnet and the other 4, to see which one is better. use the latest version of them

#base_model = InceptionV3(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
# base_model = ResNet50(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3)) #
# base_model = ResNet101(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
base_model = Xception(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
# base_model = VGG16(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))
# base_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (img_width,img_height,3))


#use resnet - try various versions of resnet
#compare googlenet to resnet and the other 4, to see which one is better. use the latest version of them

#changing the last layer
for layer in base_model.layers:
    layer.trainable = False
#https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a

# #changing the last layer
# for layer in base_model.layers:
#   layer.trainable = False



In [ ]:
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.summary()



In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, 
                    epochs = epochs, 
                    steps_per_epoch = X_train.shape[0]//batch_size, 
                    validation_data = validation_generator, 
                    validation_steps = X_val.shape[0]//batch_size, callbacks=[lrr],  verbose = 1)

In [ ]:
#Plotting the training and validation loss and accuracy
f,ax=plt.subplots(2,1) 

#Loss
ax[0].plot(model.history.history['loss'],color='b',label='Training Loss')
ax[0].plot(model.history.history['val_loss'],color='r',label='Validation Loss')

#Accuracy
ax[1].plot(model.history.history['accuracy'],color='b',label='Training  Accuracy')
ax[1].plot(model.history.history['val_accuracy'],color='r',label='Validation Accuracy')

In [ ]:
#Testing with regular
df_regular = pd.read_excel("regular.xlsx", header=None)
df_regular_label = pd.read_excel("regular.xlsx", sheet_name=1, header=None)

df_regular_label = df_regular_label.iloc[0:15 , 0:1]
df_regular_label.columns =['Class']

In [ ]:
print(df_regular.shape)
print(df_regular_label.shape)

In [ ]:
regular_path = 'scalograms/verification/regular/'

In [ ]:
ids = []
names = []

for row in df_regular.itertuples():
    
    image_id = 'image' + str(row.Index)
    name = 'image' + str(row.Index) + '.png'

    ids.append(image_id)
    names.append(name)  
        
        
        
    filepath = regular_path + name
        
    plot_wavelet(df_regular.iloc[row.Index], filepath)

In [ ]:
X_regular = extract_scalogram(regular_path)


df_regular_label['Class_int'] = pd.Categorical(df_regular_label['Class']).codes

y_regular = df_regular_label['Class_int']


In [ ]:
y_regular=to_categorical(y_regular)

print((X_regular.shape,y_regular.shape))

In [ ]:
X_regular = normalize_negative_one(X_regular)

In [ ]:
y_pred = np.argmax(model.predict(X_regular), axis=-1)

In [ ]:
print(y_pred) #predict what class the test

In [ ]:
#testing with midgrade
df_midgrade = pd.read_excel("midgrade.xlsx", header=None)
df_midgrade_label = pd.read_excel("midgrade.xlsx", sheet_name=1, header=None)

df_midgrade_label = df_midgrade_label.iloc[0:15 , 0:1]
df_midgrade_label.columns =['Class']

print(df_midgrade.shape)
print(df_midgrade_label.shape)

In [ ]:
midgrade_path = 'scalograms/verification/midgrade/'

In [ ]:
ids = []
names = []

for row in df_midgrade.itertuples():
    
    image_id = 'image' + str(row.Index)
    name = 'image' + str(row.Index) + '.png'

    ids.append(image_id)
    names.append(name)  
        
        
        
    filepath = midgrade_path + name
        
    plot_wavelet(df_midgrade.iloc[row.Index], filepath)

In [ ]:
X_midgrade = extract_scalogram(midgrade_path)


df_midgrade_label['Class_int'] = pd.Categorical(df_midgrade_label['Class']).codes

y_midgrade = df_midgrade_label['Class_int']

In [ ]:
y_midgrade=to_categorical(y_midgrade)

print((X_midgrade.shape,y_midgrade.shape))

In [ ]:
X_midgrade = normalize_negative_one(X_midgrade)

In [ ]:
y_pred = np.argmax(model.predict(X_midgrade), axis=-1)

In [ ]:
print(y_pred)

In [ ]:
#testing with premium
df_premium = pd.read_excel("premium.xlsx", header=None)
df_premium_label = pd.read_excel("premium.xlsx", sheet_name=1, header=None)

df_premium_label = df_premium_label.iloc[0:15 , 0:1]
df_premium_label.columns =['Class']

print(df_premium.shape)
print(df_premium_label.shape)

In [ ]:
premium_path = 'scalograms/verification/premium/'

In [ ]:
ids = []
names = []

for row in df_premium.itertuples():
    
    image_id = 'image' + str(row.Index)
    name = 'image' + str(row.Index) + '.png'

    ids.append(image_id)
    names.append(name)  
        
        
        
    filepath = premium_path + name
        
    plot_wavelet(df_premium.iloc[row.Index], filepath)

In [ ]:
X_premium = extract_scalogram(premium_path)


df_premium_label['Class_int'] = pd.Categorical(df_premium_label['Class']).codes

y_premium = df_premium_label['Class_int']

In [ ]:
y_premium=to_categorical(y_premium)

print((X_premium.shape,y_premium.shape))

In [ ]:
X_premium = normalize_negative_one(X_premium)

In [ ]:
y_pred = np.argmax(model.predict(X_premium), axis=-1)
print(y_pred)

In [ ]:
df_weathered = pd.read_excel("weathered gasoline.xlsx", header=None)
df_weathered_label = pd.read_excel("weathered gasoline.xlsx", sheet_name=1, header=None)

In [ ]:
df_weathered_label = df_weathered_label.iloc[: , 0:2]
df_weathered_label.columns =['Class', 'Values']

In [ ]:
df1 = pd.concat([df_weathered, df_weathered_label], axis=1)

In [ ]:
df1.shape

In [ ]:
df1.head(5)

In [ ]:
df_weathered_25 = df1[df1.Values == 0.25]

df_weathered_50 = df1[df1.Values == 0.5]

In [ ]:
df_weathered_25_label = df_weathered_25['Class']

df_weathered_25.drop(['Class', 'Values'], axis=1, inplace=True)
df_weathered_25.head(10)

In [ ]:
df_weathered_50_label = df_weathered_50['Class']

df_weathered_50.drop(['Class', 'Values'], axis=1, inplace=True)
df_weathered_50.head(10)

In [ ]:
weathered_25_path = 'scalograms/evaluation/25/'
weathered_50_path = 'scalograms/evaluation/50/'

In [ ]:
ids = []
names = []

for row in df_weathered_25.itertuples():
    
    image_id = 'image' + str(row.Index)
    name = 'image' + str(row.Index) + '.png'

    ids.append(image_id)
    names.append(name)  
        
        
        
    filepath = weathered_25_path + name
        
    plot_wavelet(df_weathered_25.iloc[row.Index], filepath)
    
for row in df_weathered_50.itertuples():
    
    image_id = 'image' + str(row.Index)
    name = 'image' + str(row.Index) + '.png'

    ids.append(image_id)
    names.append(name)  
        
        
        
    filepath = weathered_50_path + name
        
    plot_wavelet(df_weathered_50.iloc[row.Index], filepath)

In [ ]:
X_weathered_25 = extract_scalogram(weathered_25_path)
X_weathered_25 = normalize_negative_one(X_weathered_25)


df_weathered_25_label['Class_int'] = pd.Categorical(df_weathered_25_label['Class']).codes
y_weathered_25 = df_weathered_25_label['Class_int']
y_weathered_25 = to_categorical(y_weathered_25)

In [ ]:
y_pred = np.argmax(model.predict(X_weathered_25), axis=-1)
print(y_pred)

In [ ]:
X_weathered_50 = extract_scalogram(weathered_50_path)
X_weathered_50 = normalize_negative_one(X_weathered_50)


df_weathered_50_label['Class_int'] = pd.Categorical(df_weathered_50_label['Class']).codes
y_weathered_50 = df_weathered_50_label['Class_int']
y_weathered_50 = to_categorical(y_weathered_50)

In [ ]:
y_pred = np.argmax(model.predict(X_weathered_50), axis=-1)
print(y_pred)